In [ ]:
import pandas as pd
import numpy as np
import os
import shutil
import librosa
from sklearn.utils import shuffle

pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 50) 

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
newdata = pd.read_csv('/content/drive/My Drive/project/annotations_final.csv', sep="\t")
newdata.head(5)

,clip_id,no voice,singer,duet,plucking,hard rock,world,bongos,harpsichord,female singing,clasical,sitar,chorus,female opera,male vocal,vocals,clarinet,heavy,silence,beats,men,woodwind,funky,no strings,chimes,foreign,no piano,horns,classical,female,no voices,soft rock,eerie,spacey,jazz,guitar,quiet,no beat,banjo,electric,solo,violins,folk,female voice,wind,happy,ambient,new age,synth,funk,no singing,middle eastern,trumpet,percussion,drum,airy,voice,repetitive,birds,space,strings,bass,harpsicord,medieval,male voice,girl,keyboard,acoustic,loud,classic,string,drums,electronic,not classical,chanting,no violin,not rock,no guitar,organ,no vocal,talking,choral,weird,opera,soprano,fast,acoustic guitar,electric guitar,male singer,man singing,classical guitar,country,violin,electro,reggae,tribal,dark,male opera,no vocals,irish,electronica,horn,operatic,arabic,lol,low,instrumental,trance,chant,strange,drone,synthesizer,heavy metal,modern,disco,bells,man,deep,fast beat,industrial,hard,harp,no flute,jungle,pop,lute,female vocal,oboe,mellow,orchestral,viola,light,echo,piano,celtic,male vocals,orchestra,eastern,old,flutes,punk,spanish,sad,sax,slow,male,blues,vocal,indian,no singer,scary,india,woman,woman singing,rock,dance,piano solo,guitars,no drums,jazzy,singing,cello,calm,female vocals,voices,different,techno,clapping,house,monks,flute,not opera,not english,oriental,beat,upbeat,soft,noise,choir,female singer,rap,metal,hip hop,quick,water,baroque,women,fiddle,english,mp3_path
0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,f/american_bach_soloists-j_s__bach_solo_cantat...
1,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,f/american_bach_soloists-j_s__bach_solo_cantat...
2,10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,f/american_bach_soloists-j_s__bach_solo_cantat...
3,11,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,f/american_bach_soloists-j_s__bach_solo_cantat...
4,12,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,f/american_bach_soloists-j_s__bach_solo_cantat...


In [ ]:
newdata.info()
newdata.columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25863 entries, 0 to 25862
Columns: 190 entries, clip_id to mp3_path
dtypes: int64(189), object(1)
memory usage: 37.5+ MB


Index(['clip_id', 'no voice', 'singer', 'duet', 'plucking', 'hard rock',
       'world', 'bongos', 'harpsichord', 'female singing',
       ...
       'rap', 'metal', 'hip hop', 'quick', 'water', 'baroque', 'women',
       'fiddle', 'english', 'mp3_path'],
      dtype='object', length=190)

In [ ]:
clip_id = newdata[["clip_id", "mp3_path"]].as_matrix()[:,0]
mp3_path = newdata[["clip_id", "mp3_path"]].as_matrix()[:,1]

synonyms = [['beat', 'beats'],
            ['chant', 'chanting'],
            ['choir', 'choral'],
            ['classical', 'clasical', 'classic'],
            ['drum', 'drums'],
            ['electro', 'electronic', 'electronica', 'electric'],
            ['fast', 'fast beat', 'quick'],
            ['female', 'female singer', 'female singing', 'female vocals', 'female vocal', 'female voice', 'woman', 'woman singing', 'women'],
            ['flute', 'flutes'],
            ['guitar', 'guitars'],
            ['hard', 'hard rock'],
            ['harpsichord', 'harpsicord'],
            ['heavy', 'heavy metal', 'metal'],
            ['horn', 'horns'],
            ['india', 'indian'],
            ['jazz', 'jazzy'],
            ['male', 'male singer', 'male vocal', 'male vocals', 'male voice', 'man', 'man singing', 'men'],
            ['no beat', 'no drums'],
            ['no singer', 'no singing', 'no vocal','no vocals', 'no voice', 'no voices', 'instrumental'],
            ['opera', 'operatic'],
            ['orchestra', 'orchestral'],
            ['quiet', 'silence'],
            ['singer', 'singing'],
            ['space', 'spacey'],
            ['string', 'strings'],
            ['synth', 'synthesizer'],
            ['violin', 'violins'],
            ['vocal', 'vocals', 'voice', 'voices'],
            ['strange', 'weird']]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [ ]:
for synonym_list in synonyms:
    newdata[synonym_list[0]] = newdata[synonym_list].max(axis=1)
    newdata.drop(synonym_list[1:], axis=1, inplace=True)


In [ ]:
newdata.info()
newdata.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25863 entries, 0 to 25862
Columns: 136 entries, clip_id to mp3_path
dtypes: int64(135), object(1)
memory usage: 26.8+ MB


,clip_id,singer,duet,plucking,world,bongos,harpsichord,sitar,chorus,female opera,clarinet,heavy,woodwind,funky,no strings,chimes,foreign,no piano,classical,female,soft rock,eerie,jazz,guitar,quiet,no beat,banjo,solo,folk,wind,happy,ambient,new age,synth,funk,middle eastern,trumpet,percussion,drum,airy,repetitive,birds,space,bass,medieval,girl,keyboard,acoustic,loud,string,not classical,no violin,not rock,no guitar,organ,talking,opera,soprano,fast,acoustic guitar,electric guitar,classical guitar,country,violin,electro,reggae,tribal,dark,male opera,irish,horn,arabic,lol,low,trance,chant,strange,drone,modern,disco,bells,deep,industrial,hard,harp,no flute,jungle,pop,lute,oboe,mellow,viola,light,echo,piano,celtic,orchestra,eastern,old,punk,spanish,sad,sax,slow,male,blues,vocal,no singer,scary,india,rock,dance,piano solo,cello,calm,different,techno,clapping,house,monks,flute,not opera,not english,oriental,beat,upbeat,soft,noise,choir,rap,hip hop,water,baroque,fiddle,english,mp3_path
0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,f/american_bach_soloists-j_s__bach_solo_cantat...
1,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,f/american_bach_soloists-j_s__bach_solo_cantat...
2,10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,f/american_bach_soloists-j_s__bach_solo_cantat...
3,11,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,f/american_bach_soloists-j_s__bach_solo_cantat...
4,12,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,f/american_bach_soloists-j_s__bach_solo_cantat...


In [ ]:
# Drop the mp3_path tag from the dataframe
newdata.drop('mp3_path', axis=1, inplace=True)
newdata.info()

In [ ]:
root = "/content/drive/My Drive/project/"
path = root+"dataset_clip_id_mp3/"
root2 = "/content/data/"
#print(len(set(os.listdir(path))))
#print(len(set(newdata['clip_id'].values)))

#data = shuffle(newdata)
#data = data.reset_index(drop=True)
#data.to_csv('/content/annotations.csv')
#data.head()

In [ ]:
data = pd.read_csv(root+'annotations.csv')
clip_ids_all = data['clip_id']
print(len(clip_ids_all))
data.head()


25863


,Unnamed: 0,clip_id,singer,duet,plucking,world,bongos,harpsichord,sitar,chorus,female opera,clarinet,heavy,woodwind,funky,no strings,chimes,foreign,no piano,classical,female,soft rock,eerie,jazz,guitar,quiet,no beat,banjo,solo,folk,wind,happy,ambient,new age,synth,funk,middle eastern,trumpet,percussion,drum,airy,repetitive,birds,space,bass,medieval,girl,keyboard,acoustic,loud,string,not classical,no violin,not rock,no guitar,organ,talking,opera,soprano,fast,acoustic guitar,electric guitar,classical guitar,country,violin,electro,reggae,tribal,dark,male opera,irish,horn,arabic,lol,low,trance,chant,strange,drone,modern,disco,bells,deep,industrial,hard,harp,no flute,jungle,pop,lute,oboe,mellow,viola,light,echo,piano,celtic,orchestra,eastern,old,punk,spanish,sad,sax,slow,male,blues,vocal,no singer,scary,india,rock,dance,piano solo,cello,calm,different,techno,clapping,house,monks,flute,not opera,not english,oriental,beat,upbeat,soft,noise,choir,rap,hip hop,water,baroque,fiddle,english
0,0,51875,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,48429,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2,52409,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3,28251,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4,13918,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
#for id in range(25863):
#    print(clip_id[id], mp3_path[id])
#    src = root+"/music" + "/" + mp3_path[id]
#    dest = root + "/dataset_clip_id_mp3/" + str(clip_id[id]) + ".mp3"
#    shutil.copy2(src,dest)
#    print(src,dest)

2 f/american_bach_soloists-j_s__bach_solo_cantatas-01-bwv54__i_aria-30-59.mp3
/content/drive/My Drive/project//music/f/american_bach_soloists-j_s__bach_solo_cantatas-01-bwv54__i_aria-30-59.mp3 /content/drive/My Drive/project//dataset_clip_id_mp3/2.mp3
6 f/american_bach_soloists-j_s__bach_solo_cantatas-01-bwv54__i_aria-146-175.mp3
/content/drive/My Drive/project//music/f/american_bach_soloists-j_s__bach_solo_cantatas-01-bwv54__i_aria-146-175.mp3 /content/drive/My Drive/project//dataset_clip_id_mp3/6.mp3
10 f/american_bach_soloists-j_s__bach_solo_cantatas-01-bwv54__i_aria-262-291.mp3
/content/drive/My Drive/project//music/f/american_bach_soloists-j_s__bach_solo_cantatas-01-bwv54__i_aria-262-291.mp3 /content/drive/My Drive/project//dataset_clip_id_mp3/10.mp3
11 f/american_bach_soloists-j_s__bach_solo_cantatas-01-bwv54__i_aria-291-320.mp3
/content/drive/My Drive/project//music/f/american_bach_soloists-j_s__bach_solo_cantatas-01-bwv54__i_aria-291-320.mp3 /content/drive/My Drive/project//dat

In [ ]:
# Convert all the mp3 files into their corresponding mel-spectrograms (melgrams).

# Audio preprocessing function
def compute_melgram(audio_path):

    # mel-spectrogram parameters
    SR = 12000
    N_FFT = 512
    N_MELS = 96
    HOP_LEN = 256
    DURA = 29.12  # to make it 1366 frame..

    src, sr = librosa.load(audio_path, sr=SR)  # whole signal
    n_sample = src.shape[0]
    n_sample_fit = int(DURA*SR)

    if n_sample < n_sample_fit:  # if too short
        src = np.hstack((src, np.zeros((int(DURA*SR) - n_sample,))))
    elif n_sample > n_sample_fit:  # if too long
        src = src[int((n_sample-n_sample_fit)/2):int((n_sample+n_sample_fit)/2)]
    logam = librosa.amplitude_to_db
    melgram = librosa.feature.melspectrogram
    ret = logam(melgram(y=src, sr=SR, hop_length=HOP_LEN,
                        n_fft=N_FFT, n_mels=N_MELS)**2,
                ref=1.0)
    ret = ret[np.newaxis, :]
    ret = np.swapaxes(ret, 1, 2)
    return ret



In [ ]:
start = 863
end = 1863
cur_batch = 1

while cur_batch <= 25:

    # Variable to save the mp3 files that don't work
    files_that_dont_work=[]
    os.chdir(root + '/dataset_clip_id_mp3/')
    specs_clip_ids = [] #ch
    dataset = np.zeros((0, 1366, 96)) #ch
    arr2 = []
    for ele in clip_ids_all:
      arr2.append(str(ele)+".mp3")

    arr3 = arr2[start:end]
    name = "batch"+str(cur_batch)


    for audio in arr3:
        #audio_paths.append(os.path.abspath(fname))
        #if os.path.isfile(root + 'dataset_clip_id_melgram/' + str(os.path.splitext(audio_path)[0]) + '.npy'):
        #    print("exist!")
        #    continue    
                #if str(os.path.splitext(audio_path)[1]) == ".mp3":
                    try:
                        melgram = compute_melgram(root + '/dataset_clip_id_mp3/'+audio)
                        dest = root2+name
                        dataset = np.concatenate((dataset, melgram), axis = 0)
                        #current_iteration = current_iteration + 1
                        specs_clip_ids.append(int(os.path.splitext(audio)[0]))
                        #if current_iteration == batchsize:
                        #np.save(dest, melgram)
                    except EOFError:
                        files_that_dont_work.append(audio)
                        continue
    np.save(dest, dataset)
    clipIds = np.array(arr3)
    path = root+name+'_clip_ids'
    np.save(path, clipIds)
    print("files that didn't work")
    print(files_that_dont_work)
    #print(specs_clip_ids)
    #print(len(specs_clip_ids))

    saved_clip_ids = np.load(path+'.npy', allow_pickle=True)
    #print(len(saved_clip_ids))
    clips = []
    for clip in saved_clip_ids:
      clips.append(int(os.path.splitext(clip)[0]))
    #print(clips)
    #len(set(list(specs_clip_ids)).difference(clips))

    selected_data = data[start:end]

    for song in files_that_dont_work:
      selected_data = selected_data[selected_data['clip_id'] != int(os.path.splitext(song)[0])]

    selected_data.drop('Unnamed: 0', axis = 1, inplace= True)
    selected_data.drop('clip_id', axis = 1, inplace=True)


    y_values = selected_data.to_numpy()
    np.save(root2+name+'_y', y_values)

    print(name)
    print(y_values.shape)
    print(dataset.shape)

    #labels = selected_data.columns
    #np.save(root+'labels', labels)
    #print(labels)
    #len(labels)

    start = end
    end = end + 1000
    cur_batch = cur_batch+1


In [ ]:
#!rm -r /content/data

In [ ]:
labels = np.load(root+'labels.npy', allow_pickle=True)
print(labels.shape)
print(labels)

(134,)
['singer' 'duet' 'plucking' 'world' 'bongos' 'harpsichord' 'sitar'
 'chorus' 'female opera' 'clarinet' 'heavy' 'woodwind' 'funky'
 'no strings' 'chimes' 'foreign' 'no piano' 'classical' 'female'
 'soft rock' 'eerie' 'jazz' 'guitar' 'quiet' 'no beat' 'banjo' 'solo'
 'folk' 'wind' 'happy' 'ambient' 'new age' 'synth' 'funk' 'middle eastern'
 'trumpet' 'percussion' 'drum' 'airy' 'repetitive' 'birds' 'space' 'bass'
 'medieval' 'girl' 'keyboard' 'acoustic' 'loud' 'string' 'not classical'
 'no violin' 'not rock' 'no guitar' 'organ' 'talking' 'opera' 'soprano'
 'fast' 'acoustic guitar' 'electric guitar' 'classical guitar' 'country'
 'violin' 'electro' 'reggae' 'tribal' 'dark' 'male opera' 'irish' 'horn'
 'arabic' 'lol' 'low' 'trance' 'chant' 'strange' 'drone' 'modern' 'disco'
 'bells' 'deep' 'industrial' 'hard' 'harp' 'no flute' 'jungle' 'pop'
 'lute' 'oboe' 'mellow' 'viola' 'light' 'echo' 'piano' 'celtic'
 'orchestra' 'eastern' 'old' 'punk' 'spanish' 'sad' 'sax' 'slow' 'male'
 'blues' 